In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    encode_kwargs={"normalize_embeddings": True}
)

In [2]:
from langchain_postgres import PGEngine

# Replace these variable values
engine = PGEngine.from_connection_string(url="postgresql+psycopg://simon:shehr farsi@localhost/PersianPoetry")

In [4]:
from langchain_postgres import Column

VECTOR_SIZE = 1024

await engine.ainit_vectorstore_table(
    table_name='beyt_vector',
    vector_size=VECTOR_SIZE,
    metadata_columns=[
        Column("type", "VARCHAR"),
        Column("book_or_ghazal", "INTEGER"),
        Column("line", "INTEGER"),
        Column("translation", "VARCHAR"),
    ],
)

In [ ]:
from langchain_postgres import PGVectorStore

beyt_store = await PGVectorStore.create(
    engine=engine,
    table_name="beyt_vector",
    embedding_service=embeddings,
    metadata_columns=["type", "book_or_ghazal", "line", "translation"],
)

## Add beyts to pgvector

In [4]:
import pandas as pd
import psycopg2

db_host = "localhost"
db_name = "PersianPoetry"
db_user = "simon"
db_password = "shehr farsi"

try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(host=db_host, database=db_name, user=db_user, password=db_password)
    cursor = conn.cursor()

    masnavi = pd.read_sql(f"SELECT * FROM masnavi;", conn)
    ghazal = pd.read_sql(f"SELECT * FROM ghazal;", conn)

    cursor.close()
    conn.close()

except psycopg2.Error as e:
    print(f"Error connecting to PostgreSQL: {e}")

/var/folders/rp/kkdtstj57150fz8lr4ssyzjr0000gp/T/ipykernel_27925/3088733360.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  masnavi = pd.read_sql(f"SELECT * FROM masnavi;", conn)
/var/folders/rp/kkdtstj57150fz8lr4ssyzjr0000gp/T/ipykernel_27925/3088733360.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ghazal = pd.read_sql(f"SELECT * FROM ghazal;", conn)


In [8]:
from langchain_core.documents import Document
import uuid

masnavi_documents = [
    Document(
        page_content = f"{row_data['beyt1']} {row_data['beyt2']}",
        metadata = {
            "id": str(uuid.uuid4()),
            "type": "masnavi",
            "book_or_ghazal": row_data['book'],
            "line": row_data['number'],
            "translation": f"{row_data['beyt1_en']} {row_data['beyt2_en']}"
        }
    )
    for idx, (_, row_data) in enumerate(masnavi.iterrows())
]

ghazal_documents = [
    Document(
        page_content = f"{row_data['beyt1']} {row_data['beyt2']}",
        metadata = {
            "id": idx,
            "type": "ghazal",
            "book_or_ghazal": row_data['ghazal_num'],
            "line": row_data['beyt_num'],
            "translation": "",
        }
    )
    for idx, (_, row_data) in enumerate(ghazal.iterrows())
]

In [9]:
documents = masnavi_documents + ghazal_documents

### Add documents (by batch to avoid postgresql max parameters)

In [11]:
batch_size = 1000

for i in range(0, len(documents), batch_size):
    batch = documents[i:i + batch_size]
    beyt_store.add_documents(batch)

## Add index

In [ ]:
from langchain_postgres.v2.indexes import HNSWIndex

index = HNSWIndex()
await beyt_store.aapply_vector_index(index)

### Query

In [25]:
results = beyt_store.similarity_search("در این خاک", k=5)

for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

first_doc = results[0]

if first_doc.metadata['type'] == 'masnavi':
    display(masnavi.query('book == @first_doc.metadata["book_or_ghazal"] and number == @first_doc.metadata["line"]'))
elif first_doc.metadata['type'] == 'ghazal':
    display(ghazal.query('ghazal_num == @first_doc.metadata["book_or_ghazal"] and beyt_num == @first_doc.metadata["line"]'))

* در این خاک در این خاک در این مزرعه پاک بجز مهر بجز عشق دگر تخم نکاریم [{'id': 15544, 'type': 'ghazal', 'book_or_ghazal': 1475, 'line': 2, 'translation': ''}]
* کاین عالم خاک خاک ارزد آن جا که بلا کند بلایی [{'id': 29339, 'type': 'ghazal', 'book_or_ghazal': 2762, 'line': 4, 'translation': ''}]
* جان از سفر دراز آمد بر خاک در تو بازآمد [{'id': 7419, 'type': 'ghazal', 'book_or_ghazal': 709, 'line': 1, 'translation': ''}]
* این دهان خود خاکْ‌خواری آمده‌ست لیک خاکی را که آن رنگین شده‌ست [{'id': 'fcfd5821-4658-45e9-9358-51f6c48b6347', 'type': 'masnavi', 'book_or_ghazal': 6, 'line': 4705, 'translation': 'This mouth, indeed, has been an eater of earth;  but an earth that has been coloured.'}]
* بمیرید بمیرید و زین مرگ مترسید کز این خاک برآیید سماوات بگیرید [{'id': 6627, 'type': 'ghazal', 'book_or_ghazal': 636, 'line': 2, 'translation': ''}]


,ghazal_num,beyt_num,beyt1,beyt2
15544,1475,2,در این خاک در این خاک در این مزرعه پاک,بجز مهر بجز عشق دگر تخم نکاریم


In [26]:
from langchain_ollama import ChatOllama

rephraser_model = ChatOllama(model="gemma3n:e4b")
rag_llm = ChatOllama(model="gemma3n:e4b")

In [ ]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from timeit import default_timer as timer

rag_prompt_template = PromptTemplate.from_template(
    "You're a RAG system, expert in interpreting Persian poetry and passionate about Jalāl al-Dīn Muḥammad Rumi poems."
    "You answer user questions based on the documents you found."
    "First, you answer precisely the question."
    "Then, you reference all the documents you found and their metadata"
    "For example you will reference them in a list like so: "
    " - Masnavi, Book 4, line 3000:\n * Persian: کس نسازد نقشِ گرمابه و خِضابجز پیِ قصدِ صواب و ناصواب \n * English: ...\n\n"
    " - Ghazal 1475, Beyt 1:\n * Persian: بجوشید بجوشید که ما اهل شعاریم	بجز عشق بجز عشق دگر کار نداریم\n * English: ...\n\n"    
    "You always include both the original Persian version and its English translation."
    "### Question: "
    "{question}"
    "### Retrieved documents:"
    "{context}"
)

rephraser_prompt_template = PromptTemplate.from_template((
    "You're a RAG system, the vector database contain Persian Rumi Poem verses "
    "That have been given to an embedding model "
    "and the user's prompt will be compared to these embedding of poem verses by "
    "a similarity search "
    "Think how the user prompt could be improved to find verses relevant to their question "
    "Make sure to expand the user question with its unwritten objective, and "
    "if there's symbols or metaphors related to the question make sure to include them."
    "Only include the prompt, don't include your thoughts, don't include and make it one sentence only."
    "Here's the user prompt to rephrase: {question}"
))

# Define state for application
class State(TypedDict):
    question: str
    rephrased_question: str
    context: List[Document]
    answer: str
    rephrase_time_elapsed: float
    retrieve_time_elapsed: float
    generate_time_elapsed: float

def rephrase(state: State):
    start = timer()
    rephrased_prompt = rephraser_prompt_template.invoke({"question": state["question"]})
    rephrased_question = rephraser_model.invoke(rephrased_prompt)
    end = timer()

    return {
        "rephrased_question": rephrased_question.content,
        "rephrase_time_elapsed": end - start
    }

def retrieve(state: State):
    start = timer()
    retrieved_docs = beyt_store.similarity_search(state["rephrased_question"], k=5)
    end = timer()

    return {
        "context": retrieved_docs,
        "retrieve_time_elapsed": end - start
    }

def generate(state: State):
    start = timer()
    docs_content = ""

    for doc in state["context"]:
        docs_content += doc.page_content + "\n"
        docs_content += ", ".join([f"{key}: {val}" for key, val in doc.metadata.items()]) + "\n\n"

    messages = rag_prompt_template.invoke({"question": state["question"], "context": docs_content})
    response = rag_llm.invoke(messages)
    end = timer()

    return {
        "answer": response.content,
        "generate_time_elapsed": end - start
    }


# Compile application and test
graph_builder = StateGraph(State).add_sequence([rephrase, retrieve, generate])
graph_builder.add_edge(START, "rephrase")
graph = graph_builder.compile()

In [47]:
def print_response(response):
    print(f'Question: {response["question"]}')
    print(f'Rephrased as: {response["rephrased_question"]}')
    print("-" * 20)
    print(response["answer"])
    print()
    print()
    print("Elapsed times:")
    print("-" * 20)
    print(f'rephrase_time_elapsed: {response["rephrase_time_elapsed"]}')
    print(f'retrieve_time_elapsed: {response["retrieve_time_elapsed"]}')
    print(f'generate_time_elapsed: {response["generate_time_elapsed"]}')

In [48]:
response = graph.invoke({"question": "I like this verse from Parvin Etesami: ز آب چشمه و باران نمی‌شود خاموش که آتشی که در اینجاست آتش جگریست Did Mawlana write anything similar? "})
print_response(response)

Question: I like this verse from Parvin Etesami: ز آب چشمه و باران نمی‌شود خاموش که آتشی که در اینجاست آتش جگریست Did Mawlana write anything similar? 
Rephrased as: Considering the user appreciates the verse's imagery of passionate inner fire and the contrast with outward suppression, and seeks a similar sentiment from Rumi, find Rumi verses expressing intense inner passion, hidden fire, or a similar paradoxical concept of outward stillness masking profound emotion.
--------------------
Yes, there are similar themes in the works of Jalāl al-Dīn Muḥammad Rūmī. The verse by Parvin Etesami, "ز آب چشمه و باران نمی‌شود خاموش که آتشی که در اینجاست آتش جگریست" (meaning: "The spring water and the rain will not be silent, for the fire that is here is the fire of zeal"), speaks of an inner fire, a passionate fervor that cannot be suppressed. This resonates with Rumi's poetry, which often uses the metaphor of fire to describe the intensity of spiritual love and longing.

Here are some relevant ex

In [ ]:
response = graph.invoke({"question": "I heard a poem starting with 'in this earth, in this earth, in this pure farm'. Can you find it for me?"})
print_response(response)

Question: I heard a poem starting with 'in this earth, in this earth, in this pure farm'. Can you find it for me?
Rephrased as: Find Persian Rumi verses that begin with the phrase "in this earth, in this earth, in this pure farm," focusing on themes of earthly existence, purity, and spiritual cultivation.




--------------------
The phrase "in this earth, in this earth, in this pure farm" is found in the following document:

- Masnavi, Book 4, line 1762:
 * Persian: این زمین را ریعِ او خود بی‌حَد است دانه‌یی را، کمترین خود هفصد است
 * English: Verily, the produce of this earth is infinite: even the least for a single seed is seven-hundredfold.




Elapsed times:
--------------------
rephrase_time_elapsed: 2.0013440829998217
retrieve_time_elapsed: 0.1360577919995194
generate_time_elapsed: 5.2828967499999635


In [50]:
response = graph.invoke({"question": "My friend has her birthday soon, can you give me some relevant verse?"})
print_response(response)

Question: My friend has her birthday soon, can you give me some relevant verse?
Rephrased as: "Find Persian Rumi verses about celebrating life, joy, and the beauty of friendship, particularly those expressing blessings and well wishes for a loved one's upcoming birthday."
--------------------
Here are some verses from Rumi that might be suitable for your friend's birthday, focusing on themes of joy, love, and celebration.

- Ghazal 1686, Line 11:
 * Persian: هر جا رَوی بیایم، هر جا رَوَم بیایی در مرگ و زندگانی با تو خوشم، خوشَستَم
 * English: Wherever you go, I come. Wherever you go, I am with you in life and death. I am pleased with you, it is good.

- Ghazal 1402, Line 7:
 * Persian: گفتم عشق را شبی راست بگو تو کیستی گفت حیات باقیم عمر خوش مکررم
 * English: I asked love one night, "Tell me, who are you?" Life replied, "I am the remaining of your life, repeat me often."

- Ghazal 1734, Line 15:
 * Persian: شهری پر از عشق و فرح بر دست هر مستی قدح این سوی نوش آن سوی صح این جوی شیر و آن 